# Pair Programming ETL Transformación II

In [1]:
import requests
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

In [2]:
pd.options.display.max_columns = None
# pd.options.display.max_rows = None

In [8]:
df = pd.read_csv("datos/attacks_limpieza_completa.csv", index_col = 0)
df.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860


El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.

El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:

## 1.  Cargaremos los dos ficheros de datos

In [41]:
df2 = pd.read_csv('files/1.clima_paises.csv', index_col = 0)
df2.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 115, 'speed':...",26,-1,11,1012,none,0,0,110,4,Papua New Guinea
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 105, 'speed':...",26,-1,12,1013,none,0,0,105,4,Papua New Guinea


In [7]:
df3 = pd.read_csv("datos/datos_union_clima_ataques.csv", index_col = 0)
df3.head(2)

,case_number,year,type,country,area,location,activity,name,age,time,species_,injury,fecha_limpia,fatal,sex,pais,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,wind_950mb,wind_900mb,wind_850mb,wind_800mb,wind_750mb,wind_700mb,wind_650mb,wind_600mb,wind_550mb,wind_500mb,wind_450mb,wind_400mb,wind_350mb,wind_300mb,wind_250mb,wind_200mb,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,2018.06.25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,57,18h00,White shark,"No injury to occupant, outrigger canoe and pad...",Jun,N,F,USA,4.953125,6.15625,6.8125,6.296875,5.03125,5.296875,5.234375,5.09375,5.078125,6.015625,6.5,6.84375,7.296875,5.265625,4.015625,-0.3125,3.125,3.375,3.6875,3.75,3.609375,3.625,4.046875,4.5625,5.140625,5.671875,6.1875,6.59375,7.0,7.546875,7.90625,7.21875,31.5,97.5,5.078125,-9999.0,-9999.0,-9999.0,11.421875,9.6875,4.25,1013.703125,1.234375,0.0,192.109375,2.921875
1,2018.06.18,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonÂ McNeely,11,14h00 -15h00,NaN,Minor injury to left thigh,Jun,N,F,USA,4.953125,6.15625,6.8125,6.296875,5.03125,5.296875,5.234375,5.09375,5.078125,6.015625,6.5,6.84375,7.296875,5.265625,4.015625,-0.3125,3.125,3.375,3.6875,3.75,3.609375,3.625,4.046875,4.5625,5.140625,5.671875,6.1875,6.59375,7.0,7.546875,7.90625,7.21875,31.5,97.5,5.078125,-9999.0,-9999.0,-9999.0,11.421875,9.6875,4.25,1013.703125,1.234375,0.0,192.109375,2.921875


## 2. Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:
USA
Australia
New Zealand
South Africa
Papua New Guinea

In [9]:
df["country"].unique()

array(['usa', 'australia', 'mexico', 'brazil', 'england', 'south africa',
       'thailand', 'costa rica', 'maldives', 'bahamas', 'new caledonia',
       'ecuador', 'malaysia', 'libya', 'Unknown', 'cuba', 'mauritius',
       'new zealand', 'spain', 'samoa', 'solomon islands', 'japan',
       'egypt', 'st helena, british overseas territory', 'comoros',
       'reunion', 'french polynesia', 'united kingdom',
       'united arab emirates', 'philippines', 'indonesia', 'china',
       'columbia', 'cape verde', 'fiji', 'dominican republic',
       'cayman islands', 'aruba', 'mozambique', 'puerto rico', 'italy',
       'atlantic ocean', 'greece', 'st. martin', 'france',
       'papua new guinea', 'trinidad & tobago', 'kiribati', 'israel',
       'diego garcia', 'taiwan', 'jamaica', 'palestinian territories',
       'guam', 'seychelles', 'belize', 'nigeria', 'tonga', 'scotland',
       'canada', 'croatia', 'saudi arabia', 'chile', 'antigua', 'kenya',
       'russia', 'turks & caicos', 'united 

In [10]:
df_ataques = df[(df["country"]== 'usa') | 
            (df["country"]== "australia") | 
            (df["country"]== "new zealand") |
            (df["country"]== "south africa") |
            (df["country"]== "papua new guinea")
             ]

In [11]:
df_ataques.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860


In [12]:
df_ataques["country"].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

In [13]:
df_ataques.shape

(4335, 21)

## 3. Del dataframe de los datos climáticos seleccionaremos todas las columnas.

In [14]:
df2.columns

Index(['timepoint', 'cloudcover', 'highcloud', 'midcloud', 'lowcloud',
       'rh_profile', 'wind_profile', 'temp2m', 'lifted_index', 'rh2m',
       'msl_pressure', 'prec_type', 'prec_amount', 'snow_depth',
       'wind10m.direction', 'wind10m.speed', 'pais'],
      dtype='object')

## 4. Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
4.1 Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.

In [15]:
df2.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 210, 'speed':...",12,15,4,1026,none,0,0,270,2,USA
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 345, 'speed':...",12,15,3,1027,none,0,0,15,2,USA


In [16]:
df2.reset_index()

,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 210, 'speed':...",12,15,4,1026,none,0,0,270,2,USA
1,1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 345, 'speed':...",12,15,3,1027,none,0,0,15,2,USA
2,2,9,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 180, 'speed':...",12,15,2,1028,none,0,0,310,2,USA
3,3,12,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 270, 'speed':...",14,15,1,1028,none,0,0,60,2,USA
4,4,15,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 180, 'speed':...",14,15,3,1027,none,0,0,85,2,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,315,180,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 75, 'speed': ...",25,-1,11,1010,rain,3,0,75,4,Papua New Guinea
316,316,183,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 85, 'speed': ...",26,-1,11,1011,rain,3,0,80,4,Papua New Guinea
317,317,186,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 15}, {'layer': '900m...","[{'layer': '950mb', 'direction': 85, 'speed': ...",26,-1,12,1013,rain,3,0,85,4,Papua New Guinea
318,318,189,2,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 15}, {'layer': '900m...","[{'layer': '950mb', 'direction': 85, 'speed': ...",26,-1,12,1011,rain,3,0,90,4,Papua New Guinea


In [17]:
df2.shape

(320, 17)

In [18]:
df2.dtypes

timepoint             int64
cloudcover            int64
highcloud             int64
midcloud              int64
lowcloud              int64
rh_profile           object
wind_profile         object
temp2m                int64
lifted_index          int64
rh2m                  int64
msl_pressure          int64
prec_type            object
prec_amount           int64
snow_depth            int64
wind10m.direction     int64
wind10m.speed         int64
pais                 object
dtype: object

In [19]:
import ast

In [20]:
df2['wind_profile']= df2['wind_profile'].apply(ast.literal_eval)

In [21]:
df2.head

<bound method NDFrame.head of      timepoint  cloudcover  highcloud  midcloud  lowcloud  \
0            3           1      -9999     -9999     -9999   
1            6           1      -9999     -9999     -9999   
2            9           1      -9999     -9999     -9999   
3           12           1      -9999     -9999     -9999   
4           15           1      -9999     -9999     -9999   
..         ...         ...        ...       ...       ...   
315        180           7      -9999     -9999     -9999   
316        183           6      -9999     -9999     -9999   
317        186           6      -9999     -9999     -9999   
318        189           2      -9999     -9999     -9999   
319        192           2      -9999     -9999     -9999   

                                            rh_profile  \
0    [{'layer': '950mb', 'rh': 3}, {'layer': '900mb...   
1    [{'layer': '950mb', 'rh': 2}, {'layer': '900mb...   
2    [{'layer': '950mb', 'rh': 2}, {'layer': '900mb...   
3    

In [22]:
# una vez que tengamos la columna cambiada, una fantasía de Pandas es que si hago un apply sobre una columna 
# cuyos valores son diccionarios o listas nos va a genererar una columna con los valores de los diccionarios o listas. 
# Donde cada columna será key del diccionario o cada elemento de la lista. 

y = df2['wind_profile'].apply(pd.Series)
y.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': 210, 'speed': 2}","{'layer': '900mb', 'direction': 250, 'speed': 2}","{'layer': '850mb', 'direction': 300, 'speed': 3}","{'layer': '800mb', 'direction': 305, 'speed': 4}","{'layer': '750mb', 'direction': 315, 'speed': 4}","{'layer': '700mb', 'direction': 320, 'speed': 5}","{'layer': '650mb', 'direction': 315, 'speed': 5}","{'layer': '600mb', 'direction': 310, 'speed': 5}","{'layer': '550mb', 'direction': 305, 'speed': 5}","{'layer': '500mb', 'direction': 305, 'speed': 6}","{'layer': '450mb', 'direction': 310, 'speed': 6}","{'layer': '400mb', 'direction': 325, 'speed': 6}","{'layer': '350mb', 'direction': 330, 'speed': 7}","{'layer': '300mb', 'direction': 335, 'speed': 7}","{'layer': '250mb', 'direction': 335, 'speed': 8}","{'layer': '200mb', 'direction': 340, 'speed': 8}"
1,"{'layer': '950mb', 'direction': 345, 'speed': 2}","{'layer': '900mb', 'direction': 35, 'speed': 2}","{'layer': '850mb', 'direction': 355, 'speed': 3}","{'layer': '800mb', 'direction': 325, 'speed': 3}","{'layer': '750mb', 'direction': 315, 'speed': 3}","{'layer': '700mb', 'direction': 310, 'speed': 4}","{'layer': '650mb', 'direction': 310, 'speed': 5}","{'layer': '600mb', 'direction': 305, 'speed': 5}","{'layer': '550mb', 'direction': 305, 'speed': 5}","{'layer': '500mb', 'direction': 310, 'speed': 5}","{'layer': '450mb', 'direction': 325, 'speed': 6}","{'layer': '400mb', 'direction': 330, 'speed': 7}","{'layer': '350mb', 'direction': 325, 'speed': 7}","{'layer': '300mb', 'direction': 325, 'speed': 7}","{'layer': '250mb', 'direction': 335, 'speed': 7}","{'layer': '200mb', 'direction': 340, 'speed': 8}"
2,"{'layer': '950mb', 'direction': 180, 'speed': 2}","{'layer': '900mb', 'direction': 0, 'speed': 2}","{'layer': '850mb', 'direction': 330, 'speed': 3}","{'layer': '800mb', 'direction': 335, 'speed': 4}","{'layer': '750mb', 'direction': 330, 'speed': 4}","{'layer': '700mb', 'direction': 330, 'speed': 5}","{'layer': '650mb', 'direction': 335, 'speed': 5}","{'layer': '600mb', 'direction': 330, 'speed': 5}","{'layer': '550mb', 'direction': 320, 'speed': 5}","{'layer': '500mb', 'direction': 325, 'speed': 6}","{'layer': '450mb', 'direction': 325, 'speed': 6}","{'layer': '400mb', 'direction': 320, 'speed': 6}","{'layer': '350mb', 'direction': 320, 'speed': 6}","{'layer': '300mb', 'direction': 335, 'speed': 6}","{'layer': '250mb', 'direction': 330, 'speed': 6}","{'layer': '200mb', 'direction': 345, 'speed': 7}"
3,"{'layer': '950mb', 'direction': 270, 'speed': 2}","{'layer': '900mb', 'direction': 285, 'speed': 2}","{'layer': '850mb', 'direction': 305, 'speed': 3}","{'layer': '800mb', 'direction': 320, 'speed': 4}","{'layer': '750mb', 'direction': 325, 'speed': 4}","{'layer': '700mb', 'direction': 330, 'speed': 5}","{'layer': '650mb', 'direction': 335, 'speed': 5}","{'layer': '600mb', 'direction': 335, 'speed': 5}","{'layer': '550mb', 'direction': 330, 'speed': 5}","{'layer': '500mb', 'direction': 325, 'speed': 5}","{'layer': '450mb', 'direction': 325, 'speed': 5}","{'layer': '400mb', 'direction': 315, 'speed': 5}","{'layer': '350mb', 'direction': 300, 'speed': 5}","{'layer': '300mb', 'direction': 300, 'speed': 6}","{'layer': '250mb', 'direction': 320, 'speed': 5}","{'layer': '200mb', 'direction': 335, 'speed': 6}"
4,"{'layer': '950mb', 'direction': 180, 'speed': 2}","{'layer': '900mb', 'direction': 280, 'speed': 3}","{'layer': '850mb', 'direction': 295, 'speed': 3}","{'layer': '800mb', 'direction': 310, 'speed': 4}","{'layer': '750mb', 'direction': 310, 'speed': 4}","{'layer': '700mb', 'direction': 315, 'speed': 4}","{'layer': '650mb', 'direction': 315, 'speed': 4}","{'layer': '600mb', 'direction': 315, 'speed': 5}","{'layer': '550mb', 'direction': 325, 'speed': 5}","{'layer': '500mb', 'direction': 330, 'speed': 5}","{'layer': '450mb', 'direction': 330, 'speed': 5}","{'layer': '400mb', 'direction': 320, 'speed': 5}","{'layer': '350mb', 'direction': 320, 'speed': 5}","{'layer': '300mb', 'direct

In [23]:
x3 = df2["rh_profile"].apply(pd.Series)
x3.head()

,0
0,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb..."
1,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb..."
2,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb..."
3,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb..."
4,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb..."


Columna wind, se saca los valores para velocidad y dirección

In [24]:
# Por eso empezamos con un for para iterar por cada una de las columnas. 
for i in range(len(y.columns)): 

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "wind_dir_" + str(y[i].apply(pd.Series)["layer"][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(y[i].apply(pd.Series)["direction"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df2.insert(i, nombre, valores)

# una vez que hayamos hecho esto para las dos columnas ya podremos hacer el gropuby para después unir toda la información. 

In [25]:
df2.head(3)

,wind_dir_950mb,wind_dir_900mb,wind_dir_850mb,wind_dir_800mb,wind_dir_750mb,wind_dir_700mb,wind_dir_650mb,wind_dir_600mb,wind_dir_550mb,wind_dir_500mb,wind_dir_450mb,wind_dir_400mb,wind_dir_350mb,wind_dir_300mb,wind_dir_250mb,wind_dir_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,210,250,300,305,315,320,315,310,305,305,310,325,330,335,335,340,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 210, 'speed':...",12,15,4,1026,none,0,0,270,2,USA
1,345,35,355,325,315,310,310,305,305,310,325,330,325,325,335,340,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 345, 'speed':...",12,15,3,1027,none,0,0,15,2,USA
2,180,0,330,335,330,330,335,330,320,325,325,320,320,335,330,345,9,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 180, 'speed':...",12,15,2,1028,none,0,0,310,2,USA


Columna rh_profile

In [26]:
df2["rh_profile"].iloc[0]

"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb', 'rh': 0}, {'layer': '850mb', 'rh': -2}, {'layer': '800mb', 'rh': -1}, {'layer': '750mb', 'rh': 0}, {'layer': '700mb', 'rh': 2}, {'layer': '650mb', 'rh': 4}, {'layer': '600mb', 'rh': 6}, {'layer': '550mb', 'rh': 7}, {'layer': '500mb', 'rh': 7}, {'layer': '450mb', 'rh': 5}, {'layer': '400mb', 'rh': 1}, {'layer': '350mb', 'rh': -1}, {'layer': '300mb', 'rh': 2}, {'layer': '250mb', 'rh': 2}, {'layer': '200mb', 'rh': 4}]"

In [27]:
df2["rh_profile3"] = dict(df2["rh_profile"].str.split(pat=","))

In [28]:
df2["rh_profile3"].value_counts()

[[{'layer': '950mb',  'rh': 3},  {'layer': '900mb',  'rh': 0},  {'layer': '850mb',  'rh': -2},  {'layer': '800mb',  'rh': -1},  {'layer': '750mb',  'rh': 0},  {'layer': '700mb',  'rh': 2},  {'layer': '650mb',  'rh': 4},  {'layer': '600mb',  'rh': 6},  {'layer': '550mb',  'rh': 7},  {'layer': '500mb',  'rh': 7},  {'layer': '450mb',  'rh': 5},  {'layer': '400mb',  'rh': 1},  {'layer': '350mb',  'rh': -1},  {'layer': '300mb',  'rh': 2},  {'layer': '250mb',  'rh': 2},  {'layer': '200mb',  'rh': 4}]]             1
[[{'layer': '950mb',  'rh': 2},  {'layer': '900mb',  'rh': 0},  {'layer': '850mb',  'rh': 0},  {'layer': '800mb',  'rh': 1},  {'layer': '750mb',  'rh': 3},  {'layer': '700mb',  'rh': 5},  {'layer': '650mb',  'rh': 7},  {'layer': '600mb',  'rh': 7},  {'layer': '550mb',  'rh': 5},  {'layer': '500mb',  'rh': 3},  {'layer': '450mb',  'rh': 2},  {'layer': '400mb',  'rh': 0},  {'layer': '350mb',  'rh': 4},  {'layer': '300mb',  'rh': 5},  {'layer': '250mb',  'rh': 7},  {'layer': '200mb',

In [35]:
df2_rh = df2["rh_profile"].str.split(pat="," , n=-1 , expand=True)

In [36]:
df2_rh.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,[{'layer': '950mb','rh': 3},{'layer': '900mb','rh': 0},{'layer': '850mb','rh': -2},{'layer': '800mb','rh': -1},{'layer': '750mb','rh': 0},{'layer': '700mb','rh': 2},{'layer': '650mb','rh': 4},{'layer': '600mb','rh': 6},{'layer': '550mb','rh': 7},{'layer': '500mb','rh': 7},{'layer': '450mb','rh': 5},{'layer': '400mb','rh': 1},{'layer': '350mb','rh': -1},{'layer': '300mb','rh': 2},{'layer': '250mb','rh': 2},{'layer': '200mb','rh': 4}]
1,[{'layer': '950mb','rh': 2},{'layer': '900mb','rh': 0},{'layer': '850mb','rh': 0},{'layer': '800mb','rh': 1},{'layer': '750mb','rh': 3},{'layer': '700mb','rh': 5},{'layer': '650mb','rh': 7},{'layer': '600mb','rh': 7},{'layer': '550mb','rh': 5},{'layer': '500mb','rh': 3},{'layer': '450mb','rh': 2},{'layer': '400mb','rh': 0},{'layer': '350mb','rh': 4},{'layer': '300mb','rh': 5},{'layer': '250mb','rh': 7},{'layer': '200mb','rh': 7}]
2,[{'layer': '950mb','rh': 2},{'layer': '900mb','rh': 1},{'layer': '850mb','rh': 4},{'layer': '800mb','rh': 6},{'layer': '750mb','rh': 6},{'layer': '700mb','rh': 6},{'layer': '650mb','rh': 7},{'layer': '600mb','rh': 7},{'layer': '550mb','rh': 6},{'layer': '500mb','rh': 4},{'layer': '450mb','rh': 1},{'layer': '400mb','rh': -1},{'layer': '350mb','rh': 1},{'layer': '300mb','rh': 3},{'layer': '250mb','rh': 6},{'layer': '200mb','rh': 5}]
3,[{'layer': '950mb','rh': 1},{'layer': '900mb','rh': 2},{'layer': '850mb','rh': 5},{'layer': '800mb','rh': 5},{'layer': '750mb','rh': 3},{'layer': '700mb','rh': 3},{'layer': '650mb','rh': 3},{'layer': '600mb','rh': 5},{'layer': '550mb','rh': 6},{'layer': '500mb','rh': 6},{'layer': '450mb','rh': 2},{'layer': '400mb','rh': -1},{'layer': '350mb','rh': 0},{'layer': '300mb','rh': 11},{'layer': '250mb','rh': 1},{'layer': '200mb','rh': 5}]
4,[{'layer': '950mb','rh': 1},{'layer': '900mb','rh': 4},{'layer': '850mb','rh': 5},{'layer': '800mb','rh': 5},{'layer': '750mb','rh': 3},{'layer': '700mb','rh': 2},{'layer': '650mb','rh': 2},{'layer': '600mb','rh': 2},{'layer': '550mb','rh': 3},{'layer': '500mb','rh': 4},{'layer': '450mb','rh': 6},{'layer': '400mb','rh': 5},{'layer': '350mb','rh': 4},{'layer': '300mb','rh': 0},{'layer': '250mb','rh': 1},{'layer': '200mb','rh': 5}]


In [39]:
x = df2['rh_profile3'].apply(pd.Series)
x.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,[{'layer': '950mb','rh': 3},{'layer': '900mb','rh': 0},{'layer': '850mb','rh': -2},{'layer': '800mb','rh': -1},{'layer': '750mb','rh': 0},{'layer': '700mb','rh': 2},{'layer': '650mb','rh': 4},{'layer': '600mb','rh': 6},{'layer': '550mb','rh': 7},{'layer': '500mb','rh': 7},{'layer': '450mb','rh': 5},{'layer': '400mb','rh': 1},{'layer': '350mb','rh': -1},{'layer': '300mb','rh': 2},{'layer': '250mb','rh': 2},{'layer': '200mb','rh': 4}]
1,[{'layer': '950mb','rh': 2},{'layer': '900mb','rh': 0},{'layer': '850mb','rh': 0},{'layer': '800mb','rh': 1},{'layer': '750mb','rh': 3},{'layer': '700mb','rh': 5},{'layer': '650mb','rh': 7},{'layer': '600mb','rh': 7},{'layer': '550mb','rh': 5},{'layer': '500mb','rh': 3},{'layer': '450mb','rh': 2},{'layer': '400mb','rh': 0},{'layer': '350mb','rh': 4},{'layer': '300mb','rh': 5},{'layer': '250mb','rh': 7},{'layer': '200mb','rh': 7}]
2,[{'layer': '950mb','rh': 2},{'layer': '900mb','rh': 1},{'layer': '850mb','rh': 4},{'layer': '800mb','rh': 6},{'layer': '750mb','rh': 6},{'layer': '700mb','rh': 6},{'layer': '650mb','rh': 7},{'layer': '600mb','rh': 7},{'layer': '550mb','rh': 6},{'layer': '500mb','rh': 4},{'layer': '450mb','rh': 1},{'layer': '400mb','rh': -1},{'layer': '350mb','rh': 1},{'layer': '300mb','rh': 3},{'layer': '250mb','rh': 6},{'layer': '200mb','rh': 5}]
3,[{'layer': '950mb','rh': 1},{'layer': '900mb','rh': 2},{'layer': '850mb','rh': 5},{'layer': '800mb','rh': 5},{'layer': '750mb','rh': 3},{'layer': '700mb','rh': 3},{'layer': '650mb','rh': 3},{'layer': '600mb','rh': 5},{'layer': '550mb','rh': 6},{'layer': '500mb','rh': 6},{'layer': '450mb','rh': 2},{'layer': '400mb','rh': -1},{'layer': '350mb','rh': 0},{'layer': '300mb','rh': 11},{'layer': '250mb','rh': 1},{'layer': '200mb','rh': 5}]
4,[{'layer': '950mb','rh': 1},{'layer': '900mb','rh': 4},{'layer': '850mb','rh': 5},{'layer': '800mb','rh': 5},{'layer': '750mb','rh': 3},{'layer': '700mb','rh': 2},{'layer': '650mb','rh': 2},{'layer': '600mb','rh': 2},{'layer': '550mb','rh': 3},{'layer': '500mb','rh': 4},{'layer': '450mb','rh': 6},{'layer': '400mb','rh': 5},{'layer': '350mb','rh': 4},{'layer': '300mb','rh': 0},{'layer': '250mb','rh': 1},{'layer': '200mb','rh': 5}]


In [ ]:
# Por eso empezamos con un for para iterar por cada una de las columnas. 
for i in range(len(x.columns)): 

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(x[i].apply(pd.Series)['layer'][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(x[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df2.insert(i, nombre, valores)

# una vez que hayamos hecho esto para las dos columnas ya podremos hacer el gropuby para después unir toda la información.

In [ ]:
for a in range(len(x.columns)): 

    for k,v in x3.items():
        for i,j in v.items():
            print(j)

             # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(j[i].apply(pd.Series)['layer'][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(j[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df2.insert(j, nombre, valores)

[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '950mb'
[{'layer': '

AttributeError: 'str' object has no attribute 'apply'

Group by

In [ ]:
lista_columnas= df2.columns.tolist()
lista_columnas

['timepoint',
 'cloudcover',
 'highcloud',
 'midcloud',
 'lowcloud',
 'rh_profile',
 'wind_profile',
 'temp2m',
 'lifted_index',
 'rh2m',
 'msl_pressure',
 'prec_type',
 'prec_amount',
 'snow_depth',
 'wind10m.direction',
 'wind10m.speed',
 'pais']

In [ ]:
df_clima= df2.groupby("pais")['timepoint', 
                                'cloudcover',
                                'highcloud',
                                'midcloud',
                                'lowcloud',
                                'temp2m',
                                'lifted_index',
                                'rh2m',
                                'msl_pressure',
                                'prec_type',
                                'prec_amount',
                                'snow_depth',
                                'wind10m.direction',
                                'wind10m.speed',
                                'pais'].mean().reset_index()

/tmp/ipykernel_762/2865376009.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_clima= df2.groupby("pais")['timepoint',
/tmp/ipykernel_762/2865376009.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_clima= df2.groupby("pais")['timepoint',


In [ ]:
df_clima.head()

,pais,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,Papua New Guinea,97.5,3.546875,-9999.0,-9999.0,-9999.0,25.890625,-0.671875,11.500000,1010.265625,2.109375,0.0,84.375000,3.156250
1,Australia,97.5,3.703125,-9999.0,-9999.0,-9999.0,25.890625,-3.750000,10.906250,1014.906250,2.859375,0.0,122.968750,3.296875
2,New Zealand,97.5,6.734375,-9999.0,-9999.0,-9999.0,15.062500,9.093750,10.859375,1015.500000,3.203125,0.0,149.453125,3.359375
3,South Africa,97.5,7.031250,-9999.0,-9999.0,-9999.0,23.421875,2.562500,10.156250,1019.484375,1.781250,0.0,152.500000,2.296875
4,USA,97.5,3.562500,-9999.0,-9999.0,-9999.0,12.718750,13.265625,4.812500,1022.968750,0.000000,0.0,203.750000,2.093750


4.2 Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

In [ ]:
# os recomendamos resetear el index del dataframe de los datos climáticos para que no se repitan los nombres de las columnas.


# El primer problema al que nos podemos enfrentar es que si vemos los tipos de las columnas vemos que estas columnas son objetos, es decir, strings, lo que hará que trabajar con ellas sea un poco complicado: 
clima.dtypes

timepoint             int64
cloudcover            int64
highcloud             int64
midcloud              int64
lowcloud              int64
rh_profile           object
wind_profile         object

# en Python tenemos la librería `ast` que nos permite castear un string que dentro tiene diccionarios, o listas o tuplas a su tipo correspondiente. En nuestro caso, lo que conseguiremos es no tener strings sino listas en la columna. Esto lo haremos de la siguiente forma: 

import ast

clima['wind_profile']= clima['wind_profile'].apply(ast.literal_eval)

# una vez que tengamos la columna cambiada, una fantasía de Pandas es que si hago un apply sobre una columna cuyos valores son diccionarios o listas nos va a genererar una columna con los valores de los diccionarios o listas. Donde cada columna será key del diccionario o cada elemento de la lista. 


x = clima['wind_profile'].apply(pd.Series)


# nos creamos un dataframe nuevo con el resultado de la información de una de las columnas separadas por columnas. Esto nos va a devolver un dataframe donde cada fila será una celda del dataframe anterior. 
x = df['rh_profile'].apply(pd.Series) 

# ¿Qué es lo que ocurre cuando hacemos esto?
# Nos ha creado tantas columnas como valores tuvieramos en la lista. Donde columna es, en este caso, un diccionario (porque nuestra lista esta compuesta por distintos diccionarios)

# Ok, hemos conseguido desempaquetar la información de la lista en distintas columnas. Ahora tenemos que despempaquetar la información de los diccionarios en distintas columnas. En este caso, lo que querremos es que las key de los diccionarios sean los nombres de las columnas y los values los valores de las celdas del dataframe. Volveremos a seguir entonces la misma lógica que antes con el apply, pero en este caso necesitamos hacerlo para todo el dataframe (que es x): 

# Por eso empezamos con un for para iterar por cada una de las columnas. 
for i in range(len(x.columns)): 

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(x[i].apply(pd.Series)["layer"][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(x[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df.insert(i, nombre, valores)

# una vez que hayamos hecho esto para las dos columnas ya podremos hacer el gropuby para después unir toda la información. 